# 魔法方法
python中的魔法方法是指方法名以两个下划线开头并以两个下划线结尾的方法，因此也叫Dunder Methods (Double Underscores)。常用于运算符重载。魔法方法会在对类的某个操作时后端自动调用，而不需要自己直接调用。例如当使用+将两个数字相加时，在类内部会调用__add__()方法，再比如创建一个类A的对象，a=A()，python就会自动调用__new__和__init__。


[具体介绍](https://blog.csdn.net/itlilyer/article/details/122470719)



## 初始化和构造
1. _new_(cls, other)	是在对象实例化时第一个被调用的方法。它的主要作用是创建对象并返回对象,通常在这个方法中调用`super().__new__(cls)`来创建对象。

2. _init_(self, other)	是在对象实例化后立即被调用的初始化方法。它的作用是对已经实例化的对象进行初始化设置,为对象的使用做准备。

3. _del_(self)	对象的析构方法


__new__至少有一个参数cls,代表当前类

__new__必须要有返回值,返回创建的对象

__init__必须有一个参数self,这个是__new__的返回值, __init__不需要返回值

**new作用**
1. 可以在创建对象之前做一些校验或处理,然后决定是否需要创建这个对象。如果不需要,可以返回别的对象或直接返回None。

In [3]:
class Person:
    def __new__(cls, *args, **kwargs):
        print('__new__ called')
        return super().__new__(cls)
    
    def __init__(self, name):
        print('__init__ called')
        self.name = name
    def call_name(self):
        print(self.name)

p = Person('Jack')
p.call_name()
print('这里先调用了new然后才是init')

__new__ called
__init__ called
Jack
这里先调用了new然后才是init


实现单例模式

单例模式(Singleton Pattern)是一种设计模式,意图是让一个类只有一个实例对象。

In [7]:
class OnlyOne:
    '''单例模式'''
    __instance = None
    def __new__(cls):
        if OnlyOne.__instance is None:
            OnlyOne.__instance = super().__new__(cls)
            print('成功创建对象')
            return OnlyOne.__instance
        else:
            raise ValueError('Only one instance allowed!')
a=OnlyOne()
try:
    b=OnlyOne()
except ValueError as e:
    print(e)

成功创建对象
Only one instance allowed!


## 一元操作符
这里就体现了运算符重载的特点，我们可以让类对某些运算符做出反应，典型的例子就是列表可以相加

<table><thead><tr><th>方法名</th><th>描述</th></tr></thead><tbody><tr><td>_<em>pos</em>_(self)</td><td>会被取正操作符调用,例如 +a</td></tr><tr><td>_<em>neg</em>_(self)</td><td>会被取反操作符调用,例如 -a</td></tr><tr><td>_<em>abs</em>_(self)</td><td>在调用内置函数abs()的时候被调用, 取绝对值</td></tr><tr><td>_<em>invert</em>_(self)</td><td>在使用~操作符的时候被调用, 按位取反</td></tr><tr><td>_<em>round</em>_(self, n)</td><td>执行内置函数round()的时候被调用, 四舍五入取近似值,n为小数位数</td></tr><tr><td>_<em>floor</em>_(self)</td><td>执行内置函数math.floor()的时候被调用, 取小于等于该值的最大整数</td></tr><tr><td>_<em>ceil</em>_(self)</td><td>执行内置函数math.ceil()时被调用,取大于等于该值的最小整数</td></tr><tr><td>_<em>trunc</em>_(self)</td><td>执行math.trunc()函数时被调用, 取该值的截断整数</td></tr></tbody></table>

这里self就代表了这个类

In [22]:
class Mylist(list):
    def __init__(self, mylist):
        super().__init__(mylist)
        #继承父类的所有方法,这里self就代表类
    def __pos__(self):
        return [abs(i) for i in self]
    def __pos__(self):
        return [-i if i>=0 else i for i in self ]
    def __round__(self,n):
        # 一个实现截断的函数
        return self[:n]

a=Mylist([1,23,-2,9,-90,0])
print(a)
print(+a)
print(round(a,2))

[1, 23, -2, 9, -90, 0]
[-1, -23, -2, -9, -90, 0]
[1, 23]


## 当类表现地很像序列时，试图用len访问类长度，就可以编写__len__方法

In [24]:
class StudentName:
    def __init__(self,*names):
        self.names=names
    def __len__(self):
        return len(self.names)
    
test=StudentName('bob','jerf','zwj')
len(test)

3

## 一般如果想使用索引访问元素时
可以在类中定义这个方法（__getitem__(self, key) ），索引会被传入给这个函数

In [9]:
class Alarm:
    def __init__(self):
        self.warn='警报警报，非法访问源头：'
    def __getitem__(self,key):
        return self.warn+f'{key}'
a=Alarm()
a[9]

'警报警报，非法访问源头：9'

## 运算符重载式的魔术方法允许在执行对象加减时有不同反应

In [12]:
#例子：定义加号是计算两点之间的欧几里得距离，减号是曼哈顿距离
import math
class Coordinate:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __add__(self,other):
        '''相当于重写+，其中other为另一个对象，在执行+时被传入这里'''
        return math.sqrt((self.x-other.x)**2+(self.y-other.y)**2)
    def __sub__(self,other):
        return abs(self.x-other.x)+abs(self.y-other.y)

a=Coordinate(1,1)
b=Coordinate(0,0)

print(f'欧几里得距离为{a+b},曼哈顿距离为{a-b}')
    
        

欧几里得距离为1.4142135623730951,曼哈顿距离为2


## property是一个重要函数，可以让方法像属性一样工作
例如：我有个名单可以存储qq号和名字，如果我每次都要设置qq号和名字，再读取，就比较麻烦。我可以设置一个属性直接实现两个方法，且在删除时有提示

* 在这里 info表现地像属性，我可以获取它的值，修改它的值，删除它，而不需要改动构成它的属性来修改info

In [31]:
class Memo:
    def __init__(self,qq,name):
        self.qq=qq
        self.name=name
    def get_info(self):
        return self.qq,self.name
    def set_info(self,args):
        self.qq,self.name=args
    def delete_info(self): 
        print( f'您删除了信息{self.name}:{self.qq}')
        
    info=property(get_info,set_info,delete_info)
    '''当删除info时，第三个自动会被调用'''
a=Memo('1710269958','xiaofei')
a.info


('1710269958', 'xiaofei')

In [32]:
a.info='10086','spam'
a.qq

'10086'

In [33]:
del a.info

您删除了信息spam:10086



## 静态方法和类方法
* 类方法是类对象所拥有的方法，需要用修饰器@classmethod来标识其为类方法一般以cls作为第一个参数
* 静态方法需要通过修饰器@staticmethod来进行修饰，静态方法不需要多定义参数
* 类方法可以访问
    * 类变量：如Person.message
    * 其他类方法
* 类方法不可以访问
    * 实例方法:带self的
    * 实例变量
* 可以被调用
    * 类调用：Person.info()  （即使在类中）
    * 实例调用：实例名.info()  

* 静态方法可以访问
    * 类变量
    * 其他静态方法
* 静态方法不可以访问
    * 类方法
    * 实例变量
    * 实例方法
* 可以被调用
    * 类调用：Person.info()  （即使在类中）
    * 实例调用：实例名.info()      



### 直观解释
静态方法就像进食这个动作，他不需要知道你是不是人（类变量），或者你的饮食偏好（实例变量）。所有生物都要吃饭

类方法像进食人可以吃的食物，他需要知道你是人，但不需要知道饮食偏好

In [16]:
class App:
    platform='Android'    #类变量，所有实例都有的变量
    def __init__(self,name,age):
        self.age=age
        self.name=name #创建实例变量
    def __combine(self):
        '''私有方法'''
        return {self.name:self.age}  
    @staticmethod
    def show_title():
        print('这个是一个静态方法，它无法引用类变量或实例变量,\n适合做一些不需要用它们的事情，可以节省内存')
    
    @classmethod
    def show_info(cls):
        print(App.platform)
        '''必须这样来访问类变量，而不是self或单纯变量名'''
        
app=App('zwj','19')   


In [14]:
app.show_info(),App.show_info()

Android
Android


(None, None)

In [17]:
App.show_title()

这个是一个静态方法，它无法引用类变量或实例变量,
适合做一些不需要用它们的事情，可以节省内存


### 注意这里有个问题，对于私有方法，子类不会继承

In [33]:
class App_new(App):
    def __init__(self,name,age):
        super().__init__(name,age)
    def __combine(self):
        return {self.name:self.age}
    def run(self):
        '''调用私有方法和类方法和静态方法'''
        print(self.__combine())
        App.show_info()
        App.show_title()
app=App_new('john','19')
app.run()

{'john': '19'}
Android
这个是一个静态方法，它无法引用类变量或实例变量,
适合做一些不需要用它们的事情，可以节省内存
